In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import cv2
import numpy as np

# Set the frame rate (frames per second)
frame_rate = 30
# Set the target frame count for a 8-second video
target_frame_count = 8 * frame_rate  # 8 seconds * 30 FPS = 240 frames
# Target frame size (e.g., 224x224)
target_frame_size = (224, 224)

def load_video(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)
    cap.release()
    return frames

def resample_frames(frames, target_frame_count):
    frame_count = len(frames)
    indices = np.linspace(0, frame_count - 1, target_frame_count).astype(np.int32)
    resampled_frames = [frames[i] for i in indices]
    return resampled_frames

def preprocess_frame(frame, target_size):
    # Resize frame
    frame = cv2.resize(frame, target_size)
    # Normalize pixel values to [0, 1]
    frame = frame / 255.0
    return frame

def save_video(frames, save_path, fps=30):
    if len(frames) == 0:
        return
    height, width, layers = frames[0].shape
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video = cv2.VideoWriter(save_path, fourcc, fps, (width, height))
    for frame in frames:
        frame = (frame * 255).astype(np.uint8)
        video.write(frame)
    video.release()

# Base directory containing original videos
base_dir = '/content/drive/MyDrive/Sign_language_dataset'  # Update with your base directory path
# Directory to save processed videos
processed_base_dir = '/content/drive/MyDrive/processed_sign_language_dataset'  # Update with your desired save directory path

# Process each folder (each word)
for word_folder in os.listdir(base_dir):
    word_folder_path = os.path.join(base_dir, word_folder)
    if os.path.isdir(word_folder_path):
        processed_word_folder_path = os.path.join(processed_base_dir, word_folder)
        os.makedirs(processed_word_folder_path, exist_ok=True)

        # Process each video in the word folder
        for filename in os.listdir(word_folder_path):
            if filename.endswith(".mp4"):
                video_path = os.path.join(word_folder_path, filename)
                frames = load_video(video_path)

                # Resample frames to target_frame_count
                resampled_frames = resample_frames(frames, target_frame_count)

                # Preprocess frames (resize and normalize)
                preprocessed_frames = [preprocess_frame(frame, target_frame_size) for frame in resampled_frames]

                # Debugging: Check the dimensions of the preprocessed frames
                for frame in preprocessed_frames:
                    assert frame.shape == (224, 224, 3), f"Frame shape mismatch: {frame.shape}"

                # Save processed video
                save_path = os.path.join(processed_word_folder_path, filename)
                save_video(preprocessed_frames, save_path)

print("Video preprocessing complete. Processed videos are saved in:", processed_base_dir)
